In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../util')
sys.path.append('../experiments')

In [3]:
from convert_annotations import read_webanno, resolve_ellipses, webanno_to_iob_df, EMPTY_REGEX
from pathlib import Path
from datetime import datetime
import pandas as pd

/home/Florian.Borchert/.local/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
ignore_documents = [
    '00_mundhoehlenkarzinom_0059.tsv',
    '00_mundhoehlenkarzinom_0071.tsv'
]

In [5]:
%%time
tsv_folder = '../../data_ggponc/output_ellipses/'
#tsv_folder = 'ellipses_dev'

webanno_df, sentences = read_webanno([f for f in Path(tsv_folder).glob('*.tsv') if not f.name in ignore_documents])

CPU times: user 10.1 s, sys: 653 ms, total: 10.7 s
Wall time: 12.3 s


In [6]:
%%time

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    iob_df = webanno_to_iob_df(webanno_df, 'detail', True, debug=False, collect_errors=False, skip_errors=True, all_columns=True)
    iob_df['id'] = iob_df.entity_id.where(~iob_df.entity_id.isna(), iob_df.spec_id)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49720/49720 [09:11<00:00, 90.13it/s]


CPU times: user 9min 31s, sys: 2.73 s, total: 9min 34s
Wall time: 9min 32s


In [7]:
# Anything containing a prefix, suffix or fragment is potentially an elliptical construct
ellipses = iob_df[~iob_df.prefix.str.match(EMPTY_REGEX) | ~iob_df.suffix.str.match(EMPTY_REGEX) | ~iob_df.fragment.str.match(EMPTY_REGEX)].copy()
len(ellipses)

7298

In [8]:
ellipses

token_id ts_id       span  \
file                                  sentence_id                              
00_mundhoehlenkarzinom_0002.tsv       1                   9   1-9      81-86   
                                      2                   3   2-3    166-171   
                                      5                  22  5-22    733-743   
                                      5                  23  5-23    744-749   
00_mundhoehlenkarzinom_0005.tsv       12                  9  12-9  2627-2634   
...                                                     ...   ...        ...   
29_adulte-weichgewebesarkome_0397.tsv 6                  14  6-14    851-858   
29_adulte-weichgewebesarkome_0403.tsv 1                  14  1-14    101-109   
                                      1                  15  1-15    110-120   
29_adulte-weichgewebesarkome_0404.tsv 6                   3   6-3    594-605   
29_adulte-weichgewebesarkome_0405.tsv 3                  33  3-33    539-544   

                                                         token  \
file                                  sentence_id                
00_mundhoehlenkarzinom_0002.tsv       1                  Tabak   
                                      2                  Tabak   
                                      5             gebratenem   
                                      5                  Essen   
00_mundhoehlenkarzinom_0005.tsv       12               HPV-RNA   
...                                                        ...   
29_adulte-weichgewebesarkome_0397.tsv 6                Skrotum   
29_adulte-weichgewebesarkome_0403.tsv 1               einzelne   
                                      1             Subgruppen   
29_adulte-weichgewebesarkome_0404.tsv 6            Zweitlinien   
29_adulte-weichgewebesarkome_0405.tsv 3                  Chemo   

                                                              value  \
file                                  sentence_id                     
00_mundhoehlenkarzinom_0002.tsv       1                     Finding   
                                      2                     Finding   
                                      5            Specification[9]   
                                      5            Specification[9]   
00_mundhoehlenkarzinom_0005.tsv       12                  Procedure   
...                                                             ...   
29_adulte-weichgewebesarkome_0397.tsv 6               Specification   
29_adulte-weichgewebesarkome_0403.tsv 1            Specification[3]   
                                      1            Specification[3]   
29_adulte-weichgewebesarkome_0404.tsv 6                   Procedure   
29_adulte-weichgewebesarkome_0405.tsv 3                   Procedure   

                                                                   detail  \
file                                  sentence_id                           
00_mundhoehlenkarzinom_0002.tsv       1            Diagnosis or Pathology   
                                      2            Diagnosis or Pathology   
                                      5                              *[9]   
                                      5                              *[9]   
00_mundhoehlenkarzinom_0005.tsv       12                       Diagnostic   
...                                                                   ...   
29_adulte-weichgewebesarkome_0397.tsv 6                                 *   
29_adulte-weichgewebesarkome_0403.tsv 1                              *[3]   
                                      1                              *[3]   
29_adulte-weichgewebesarkome_0404.tsv 6                       Therapeutic   
29_adulte-weichgewebesarkome_0405.tsv 3                       Therapeutic   

                                                  specified_by      prefix  \
file                                  sentence_id                            
00_mundhoehlenkarzinom_0002.tsv       1                    1-8       

In [9]:
%%time
results = resolve_ellipses(ellipses, iob_df)

CPU times: user 5min 42s, sys: 245 ms, total: 5min 42s
Wall time: 5min 42s


In [10]:
results

,file,sentence_id,full_sentence,span_index_start,span_index_end,full_span,offsets,resolution,fragment,missing_prefix,missing_suffix
0,00_mundhoehlenkarzinom_0002.tsv,1,"[Hauptrisikofaktoren, für, das, Auftreten, ein...",7,11,"[chronischer, Tabak, -, oder, Alkoholabusus]","[[69, 80], [81, 86], [86, 87], [88, 92], [93, ...","[chronischer, Tabakabusus, , oder, Alkoholabusus]",True,False,True
1,00_mundhoehlenkarzinom_0002.tsv,2,"[Bei, chronischem, Tabak, -, oder, Alkoholabus...",1,5,"[chronischem, Tabak, -, oder, Alkoholabusus]","[[4, 15], [16, 21], [21, 22], [23, 27], [28, 41]]","[chronischem, Tabakabusus, , oder, Alkoholabusus]",True,False,True
2,00_mundhoehlenkarzinom_0002.tsv,5,"[Neben, dem, Konsum, von, Tabak, oder, Alkohol...",16,22,"[übermäßiger, Konsum, von, Fleisch, oder, gebr...","[[90, 101], [102, 108], [109, 112], [113, 120]...","[übermäßiger, Konsum, von, Fleisch, oder, von ...",False,True,False
3,00_mundhoehlenkarzinom_0005.tsv,12,"[Die, hohe, Diskrepanz, zwischen, p16, Immunhi...",8,11,"[HPV-RNA, und, HPV-DNA, Nachweis]","[[55, 62], [63, 66], [67, 74], [75, 83]]","[HPV-RNA Nachweis, und, HPV-DNA, Nachweis]",True,False,True
4,00_mundhoehlenkarzinom_0005.tsv,18,"[Die, Prävalenz, von, HPV16, E6, oder, E7, Ant...",3,9,"[HPV16, E6, oder, E7, Antikörpern, in, Mundhöh...","[[18, 23], [24, 26], [27, 31], [32, 34], [35, ...","[HPV16, E6 Antikörpern, oder, HPV16 E7, Antikö...",True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
5222,29_adulte-weichgewebesarkome_0397.tsv,4,"[Sehr, viel, häufiger, wird, bei, Männern, unt...",18,22,"[Weichteiltumor, des, Skrotums, oder, Samenstr...","[[116, 130], [131, 134], [135, 143], [144, 148...","[Weichteiltumor, des, Skrotums, oder, des Same...",False,True,False
5223,29_adulte-weichgewebesarkome_0397.tsv,6,"[Klinisch, besteht, meist, eine, unilaterale, ...",4,13,"[unilaterale, ,, sich, langsam, vergrößernde, ...","[[28, 39], [39, 40], [41, 45], [46, 53], [54, ...","[unilaterale, ,, sich, langsam, vergrößernde, ...",False,True,False
5224,29_adulte-weichgewebesarkome_0403.tsv,1,"[Es, besteht, Forschungsbedarf, in, der, Kläru...",7,14,"[molekulare, Prognosemarker, für, Sarkome, ode...","[[53, 63], [64, 78], [79, 82], [83, 90], [91, ...","[molekulare, Prognosemarker, für, Sarkome, ode...",False,True,False
5225,29_adulte-weichgewebesarkome_0404.tsv,6,"[Bei, der, Zweitlinien, -, und, Folgetherapie,...",2,5,"[Zweitlinien, -, und, Folgetherapie]","[[8, 19], [19, 20], [21, 24], [25, 38]]","[Zweitlinientherapie, , und, Folgetherapie]",True,False,True


### Adding resolved sentences and metadata

In [11]:
sentence_idx = pd.DataFrame(sentences, columns=['raw_sentence'], index=webanno_df.index.drop_duplicates())

In [12]:
import math
clean_results = results.merge(right=sentence_idx, left_on=['file', 'sentence_id'], right_index=True)
clean_results['outer'] = math.nan

In [13]:
for _, k in clean_results[clean_results.duplicated(['file', 'sentence_id'])][['file', 'sentence_id']].iterrows():
    f = k['file']
    s = k['sentence_id']
    dups = clean_results[(clean_results.file == f) & (clean_results.sentence_id == s)]
    
    for i, inner in dups.iterrows():
        for j, outer in dups.iterrows():
            if i != j and inner.span_index_start >= outer.span_index_start and inner.span_index_end <= outer.span_index_end:
                if not (clean_results.loc[j, 'outer'] == i):
                    clean_results.loc[i, 'outer'] = j

In [14]:
clean_results = clean_results[clean_results.outer.isna()]

In [15]:
def resolve(row):
    out = {}
    for c in row.index:
        val = row.loc[c]
        if len(val) == 1:
            out['multi'] = False
            out[c] = val[0]
        else:
            out['multi'] = True
            if val.count(val[0]) == len(val) and c in ['full_sentence', 'raw_sentence']:
                out[c] = val[0]
            elif c in ['fragment', 'missing_prefix', 'missing_suffix']:
                out[c] = max(val)
            else:
                out[c] = val
    return pd.Series(out)

In [16]:
clean_results = clean_results.groupby(['file', 'sentence_id', 'fragment']).aggregate(tuple).applymap(list).apply(resolve, axis=1)[list(clean_results.columns.drop(['outer', 'file', 'sentence_id', 'fragment'])) + ['multi']].reset_index()

In [17]:
def get_full_res(row):
    sent = row.raw_sentence
    if row.multi:
        spans = [o for p in row.offsets for o in p]
        resolution = [r for p in row.resolution for r in p]
    else:
        spans = row.offsets
        resolution = row.resolution
    assert len(spans) == len(resolution), (spans, resolution)
    j = 0
    for span, res in zip(spans, resolution):
        start, end = span
        start += j
        end += j
        sent = sent[:start] + res + sent[end:]
        j += (start - end + len(res))
    return sent

In [18]:
clean_results['full_resolution'] = clean_results.apply(get_full_res, axis=1)

In [19]:
ellipses_sentence_count = len(results[['file', 'sentence_id']].drop_duplicates())
sentence_count = 78090
ellipses_sentence_count, sentence_count, ellipses_sentence_count / sentence_count

(4512, 78090, 0.0577794852093738)

In [20]:
print('Ellipses', len(results))
print('Forward ellipses', len(results[results.missing_prefix & ~results.missing_suffix]), len(results[results.fragment & results.missing_prefix & ~results.missing_suffix]))
print('Backward ellipses', len(results[~results.missing_prefix & results.missing_suffix]), len(results[results.fragment & ~results.missing_prefix & results.missing_suffix]))
print('Complex ellipses', len(results[results.missing_prefix & results.missing_suffix]), len(results[results.fragment & results.missing_prefix & results.missing_suffix]))

Ellipses 5227
Forward ellipses 1838 727
Backward ellipses 3052 2865
Complex ellipses 337 268


In [21]:
import random

def show_random(df, i=None):
    if i is None:
        i = random.randint(0, len(df) - 1)
    item = df.iloc[i]    
    assert len(item.full_span) == len(item.resolution)
    spans = []
    solutions = []
    marker = []
    if not item.multi:
        for span, res in zip(item.full_span, item.resolution):
            if span == res:
                marker.append(' ' * len(span))
            else:
                marker.append('^' * len(res))
            start_index = res.find(span)
            spans.append(span.rjust(start_index + len(span)).ljust(len(res) - start_index))
            solutions.append(res)
        print(' '.join(spans))
        print(' '.join(solutions))
        print(' '.join(marker))
        print(f'{item.file};{item.sentence_id};{" ".join(item.full_span)}', 'fragment:', item.fragment)
    else:
         print(f'{item.file};{item.sentence_id}')
    print(item.raw_sentence)
    print(item.full_resolution)
    print('...........')
    print('')

In [22]:
for _ in range(0, 10):
    show_random(clean_results[clean_results.multi & clean_results.fragment])

04_praevention-zervixkarzinom_0255.tsv;14
Um diese Früh- und Spätkomplikationen der Therapie einer CIN 2/3 Läsion zu reduzieren, ist die komplette Exzision nicht immer als Therapieziel anzusehen [REF].
Um diese Frühkomplikationen und Spätkomplikationen der Therapie einer CIN 2 Läsion/CIN 3 Läsion zu reduzieren, ist die komplette Exzision nicht immer als Therapieziel anzusehen [REF].
...........

09_mammakarzinom_0334.tsv;30
Bewegungstraining, vorrangig als sensomotorisches und Balance-Training sowie ergotherapeutische Therapien, Physio- und physikalische Therapie einschließlich Elektrotherapie unter Beachtung der Kontraindikationen sind daher zu empfehlen.
Bewegungstraining, vorrangig als sensomotorisches Training und Balance-Training sowie ergotherapeutische Therapien, Physiotherapie und physikalische Therapie einschließlich Elektrotherapie unter Beachtung der Kontraindikationen sind daher zu empfehlen.
...........

04_praevention-zervixkarzinom_0138.tsv;3
Bei p16/Ki-67-Negativität so

In [23]:
for _ in range(0, 10):
    show_random(clean_results[clean_results.fragment])

Ösophagus         - oder Übergangskarzinom
Ösophaguskarzinom  oder Übergangskarzinom
^^^^^^^^^^^^^^^^^                        
13_oesophaguskarzinom_0068.tsv;6;Ösophagus - oder Übergangskarzinom fragment: True
In einer Kohortenstudie erhielten 400 von 584 operierten Patienten mit einem Ösophagus- oder Übergangskarzinom eine neoadjuvante Chemotherapie [REF].
In einer Kohortenstudie erhielten 400 von 584 operierten Patienten mit einem Ösophaguskarzinom oder Übergangskarzinom eine neoadjuvante Chemotherapie [REF].
...........

Knochen           - und / oder Hirnmetastasen
Knochenmetastasen  und / oder Hirnmetastasen
^^^^^^^^^^^^^^^^^                           
21_hodentumoren_0137.tsv;1;Knochen - und / oder Hirnmetastasen fragment: True
Bei Patienten mit primär mediastinaler Erkrankung, Knochen- und/oder Hirnmetastasen und/oder inadäquatem Serumtumormarkerabfall soll eine Therapieintensivierung evaluiert werden.
Bei Patienten mit primär mediastinaler Erkrankung, Knochenmetastasen und/oder

In [24]:
#ss = clean_results[~clean_results.fragment][clean_results[~clean_results.fragment].resolution.map(lambda r: any([t for t in r if '-' in t]))]

In [25]:
#for _ in range(0, 10):
#    show_random(ss)

In [26]:
for _ in range(0, 10):
    show_random(clean_results[~clean_results.fragment])

4 Zyklen konventioneller Chemotherapie ( Dexa-BEAM ) und 2 Zyklen Dexa-BEAM gefolgt von BEAM-Hochdosischemotherapie und     autologer Stammzelltransplantation
4 Zyklen konventioneller Chemotherapie ( Dexa-BEAM ) und 2 Zyklen Dexa-BEAM gefolgt von BEAM-Hochdosischemotherapie und von autologer Stammzelltransplantation
                                                                                                                        ^^^^^^^^^^^^^                         
18_hodgkin-lymphom_0148.tsv;6;4 Zyklen konventioneller Chemotherapie ( Dexa-BEAM ) und 2 Zyklen Dexa-BEAM gefolgt von BEAM-Hochdosischemotherapie und autologer Stammzelltransplantation fragment: False
Patienten wurden zwischen 4 Zyklen konventioneller Chemotherapie (Dexa-BEAM) und 2 Zyklen Dexa-BEAM gefolgt von BEAM-Hochdosischemotherapie und autologer Stammzelltransplantation randomisiert.
Patienten wurden zwischen 4 Zyklen konventioneller Chemotherapie (Dexa-BEAM) und 2 Zyklen Dexa-BEAM gefolgt von BEAM-Hochdosische

# Train / Dev / Test Splits

In [27]:
def create_export(subset, with_fragment=False):
    subset = subset.copy()
    join_fn = lambda x: ' '.join([xi.strip() for xi in x]) if type(x[0]) == str else '; '.join([' '.join([yi.strip() for yi in y]) for y in x])
    subset['text_whitespace_tokenized'] = subset.full_span.map(join_fn)
    subset['resolved_whitespace_tokenized'] = subset.resolution.map(join_fn)
    subset = subset[['file', 'sentence_id', 'text_whitespace_tokenized', 'resolved_whitespace_tokenized', 'raw_sentence', 'full_resolution'] + (['fragment'] if with_fragment else [])]
    return subset.set_index(['file', 'sentence_id'])

In [28]:
from sklearn.model_selection import train_test_split

In [55]:
fragments = create_export(clean_results[clean_results.fragment])

In [56]:
files = list(fragments.index.levels[0].unique())

In [57]:
train_ratio = 0.7
dev_ratio = 0.15
test_ratio = 0.15

x_train, _x_test = train_test_split(files, test_size=1 - train_ratio, random_state=1)

x_dev, x_test = train_test_split(_x_test, test_size=test_ratio/(test_ratio + dev_ratio), random_state=42) 

print(len(x_train), len(x_dev), len(x_test))

1410 302 303


In [58]:
fragments.loc[x_train, 'split'] = 'train'
fragments.loc[x_dev, 'split'] = 'dev'
fragments.loc[x_test, 'split'] = 'test'
fragments.reset_index().to_csv('../../data_ggponc/ellipses/ggponc_ellipses_compounds.tsv', sep='\t', index=False)

In [59]:
controls = sentence_idx[~sentence_idx.index.isin(clean_results.set_index(['file', 'sentence_id']).index)]
controls = controls.query('file in @files')
control_idx = set([t[0] for t in controls.index])

controls.loc[list(set(x_train).intersection(control_idx)), 'split'] = 'train'
controls.loc[list(set(x_dev).intersection(control_idx)), 'split'] = 'dev'
controls.loc[list(set(x_test).intersection(control_idx)), 'split'] = 'test'
controls.reset_index().to_csv('../../data_ggponc/ellipses/ggponc_no_ellipses_all.tsv', sep='\t', index=False)
controls.reset_index().sample(len(fragments), random_state=42).to_csv('../../data_ggponc/ellipses/ggponc_no_ellipses_small.tsv', sep='\t', index=False)

In [35]:
s = '04_praevention-zervixkarzinom_0142.tsv	12'
f, s = tuple(s.split('\t'))

In [36]:
frgm = clean_results#[clean_results.fragment]
show_random(frgm[(frgm.file == f) & (frgm.sentence_id == int(s))])

HPV - 16 /     18
HPV - 16 / HPV-18
           ^^^^^^
04_praevention-zervixkarzinom_0142.tsv;12;HPV - 16 / 18 fragment: True
Testung auf HPV-16 und HPV-16/18, p16 und ProExC waren zwar deutlich spezifischer als die HPV-Basistests aber hatten eine unzureichende Sensitivität.
Testung auf HPV-16 und HPV-16/HPV-18, p16 und ProExC waren zwar deutlich spezifischer als die HPV-Basistests aber hatten eine unzureichende Sensitivität.
...........



#### All annotations covered by export?

In [63]:
fragments = clean_results[clean_results.fragment]

In [64]:
fragment_ix = set(fragments.set_index(['file', 'sentence_id']).index.drop_duplicates())
ellipses_ix = set(ellipses[~ellipses.fragment.str.match(EMPTY_REGEX)].index.drop_duplicates())

In [65]:
len(ellipses_ix - fragment_ix) == 0

True